<a href="https://colab.research.google.com/github/dani-lbnl/mudit/blob/main/TopicModel_RecEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## install and imports

In [1]:
!pip install --upgrade spacy
!python -m spacy download en_core_web_lg
!pip install -U kaleido
!pip install bertopic
# !python -m spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.2 MB 5.1 MB/s 
     |████████████████████████████████| 457 kB 66.2 MB/s 
     |████████████████████████████████| 42 kB 2.0 MB/s 
     |████████████████████████████████| 10.1 MB 35.7 MB/s 
     |████████████████████████████████| 181 kB 76.9 MB/s 
     |████████████████████████████████| 660 kB 53.6 MB/s 
     |████████████████████████████████| 58 kB 6.1 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 400.7 MB 7.0 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79.9 MB 156 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 3.3 MB/s 
     |████████████████████████████████| 79 kB 8.0 MB/s 
     |████████████████████████████████| 5.2 MB 45.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 88 kB 8.5 MB/s 
     |████████████████████████████████| 4.2 MB 47.9 MB/s 
     |████████████████████████████████| 1.2 MB 54.8 MB/s 
     |███████████

In [2]:
# from top2vec import Top2Vec
import pandas as pd
import numpy as np
import multiprocessing
import time
from google.colab import drive
import os
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

import re
import string

import nltk
nltk.download('stopwords')
import spacy

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import kaleido
nlp = spacy.load('en_core_web_lg')
multiprocessing.cpu_count()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2

## helper functions

In [3]:
import joblib
import os 

def create_path_if_not_exists(datapath):
    '''Create the new file if not exists and save the data'''

    if not os.path.exists(datapath):
        os.makedirs(datapath) 

def remove_punct_df(df):

  # remove punctuation
  # text_col = [''.join(letter for letter in word if letter not in string.punctuation) for word in df]

  filt_combined = []
  for word in word_tokenize(df):

    if word.lower() not in string.punctuation:
      # print(word)
      
      filt_combined.append(word)
  filtered_ip= " ".join(filt_combined)

  return filtered_ip

def create_input_new(df):

  # for title
  indices = df['Title'].isna()
  df.loc[indices,'Title'] = ""

  # for abstract
  indices = df['Abstract'].isna()
  df.loc[indices,'Abstract'] = ""

  # combined - title + abstract
  df['Combined'] = df['Title'] + " " + df['Abstract']

  # remove blanks
  df = df[df['Combined']!=" "]

  #keep only selected cols
  df_sel = df[['Authors','Pub Year','Research Area','Combined',"Pub TYpe"]]
  df_sel= df_sel.rename(columns={'Pub Year':'pub_year',"Research Area":"research_area","Authors":"authors","Combined":"combined","Pub TYpe":"pub_type"})

  combined = list(df_sel['combined'])

  # remove patterns
  pattern = r'<inf>|</inf>|<sup>|</sup>|inf|/inf'
  comb_clean = []
  for l in combined:
    mod_string = re.sub(pattern, '', l )
    comb_clean.append(mod_string)

  # merge back to df
  df_sel['combined'] = comb_clean

  # filter spurtious yeats
  df_sel = df_sel[df_sel['pub_year']!='12.0.1.2']

  # convert years to int
  # df_sel['pub_year'] = df_sel['pub_year'].astype(str)
  df_sel['pub_year'] = df_sel['pub_year'].astype(str).replace('\.0', '', regex=True).astype(int)
  # if year is 201, that is mistyped fom 2001
  df_sel[df_sel['pub_year']==201]['pub_year'] = 2001


  return df_sel

def lemma_spacy(df_combined):

  filt_combined = []
  for word in nlp(df_combined):
    if word.lemma_ != '-PRON-' :
      filt_combined.append(word.lemma_)

  new_df = " ".join(filt_combined)

  return new_df  

def remove_stop_df(df_combined):

  # remove stopwords

  new_df= []
  filt_combined = []
  for word in word_tokenize(df_combined):

    if word.lower() not in stopwords.words('english'):
      # print(word)
      if word.lower() == "perovskites":
        filt_combined.append("perovskite")
      else:
        filt_combined.append(word)
  filtered_ip= " ".join(filt_combined)

  return filtered_ip



def flatten(t):
    return [item for sublist in t for item in sublist]


def get_authors(input_data,rep_docs):
  tt = input_data['combined'].to_list()

  auth_list= []
  for d in rep_docs:
    ind = tt.index(d)
    auth_str = input_data.authors[ind]
    auth_el = auth_str.split(" ,")
    auth_list.append(auth_el)

  auth_list = flatten(auth_list)
  if '' in auth_list:
    auth_list.remove('')

  # unduplicate repeating authors 
  # print("Length of List:",len(auth_list))
  # print(auth_list)


  final_auth_list = list(set(auth_list))
  if '' in final_auth_list:
    final_auth_list.remove('')

  # print(final_auth_list)
  # print("Length of Final List:",len(final_auth_list))

  return final_auth_list

  
def author_all_topics(model,input_data):

  dict_df = {}
  for i in range(len(model.topics)-1):
    rep_docs = model.get_representative_docs(i)
    author_list = get_authors(input_data,rep_docs)

    dict_df[i] = author_list

  # create df with topics and authors
  author_topics = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dict_df.items() ]))

  return author_topics,dict_df



## path of files 

In [4]:
drive.mount('/content/drive')
# read all files
path = '/content/drive/MyDrive/NLP/ALS Spreadsheets/'
files = os.listdir(path)
files.sort()
files
# beamline 2.4 bad ..removed; also one more check which one
# 3.2.1 bad

Mounted at /content/drive


['Beamline_1.4.xls',
 'Beamline_10.0.1.xls',
 'Beamline_10.3.1.xls',
 'Beamline_10.3.2.xls',
 'Beamline_11.0.1.xls',
 'Beamline_11.0.2.xls',
 'Beamline_11.3.1.xls',
 'Beamline_11.3.2.xls',
 'Beamline_12.0.1.1.xls',
 'Beamline_12.0.1.2.xls',
 'Beamline_12.0.1.4.xls',
 'Beamline_12.0.2.xls',
 'Beamline_12.2.1.xls',
 'Beamline_12.2.2.xls',
 'Beamline_12.3.1.xls',
 'Beamline_12.3.2.xls',
 'Beamline_2.1.xls',
 'Beamline_3.1.1.xls',
 'Beamline_3.3.2.xls',
 'Beamline_4.0.2.xls',
 'Beamline_4.0.3.1.xls',
 'Beamline_4.0.3.2.xls',
 'Beamline_4.2.2.xls',
 'Beamline_5.0.1.xls',
 'Beamline_5.0.2.xls',
 'Beamline_5.0.3.xls',
 'Beamline_5.3.1.xls',
 'Beamline_5.3.2.1.xls',
 'Beamline_5.3.2.2.xls',
 'Beamline_5.4.xls',
 'Beamline_6.0.1.xls',
 'Beamline_6.0.2.xls',
 'Beamline_6.1.2.xls',
 'Beamline_6.3.1.xls',
 'Beamline_6.3.2.xls',
 'Beamline_7.0.1.xls',
 'Beamline_7.0.2.xls',
 'Beamline_7.3.1.xls',
 'Beamline_7.3.3.xls',
 'Beamline_8.0.1.xls',
 'Beamline_8.2.1.xls',
 'Beamline_8.2.2.xls',
 'Beamline_

## main code

In [ ]:
# which model to use - "default" or "specter"
model_use = "default"

# which beamline?
beamline = "8.3.2"



In [ ]:
start_time = time.time()
iter_version = "BERTopic_SPECTER_v3/"

for i in range(1):
# for i in range(len(files)):
  print("File:",str(i+1) + ' ' + files[i])
  # beam_name = files[i].split('xls')[0][:-1]
  beam_name = "Beamline_8.3.2.xls"
  

  df = pd.read_table(path + beam_name, on_bad_lines='skip')
  # df = pd.read_table(path + files[i], on_bad_lines='skip')


  # combine abstract and title
  input_data = create_input_new(df)

  # remove stopwords
  input_data['combined'] = input_data['combined'].apply(remove_stop_df)

  # lemmatize
  input_data['combined'] = input_data['combined'].apply(lemma_spacy)

  # remove punct
  input_data['combined'] = input_data['combined'].apply(remove_punct_df)

  # train model
  sentence_model = SentenceTransformer('allenai-specter')
  topic_model_specter = BERTopic(embedding_model=sentence_model,verbose=True)
  list_text = input_data['combined'].to_list()

  topics, probs = topic_model_specter.fit_transform(list_text)
  # print("Before removing:")
  # print(topic_model_specter.topics)

  # topic_model_specter.topics.pop(-1, None)
  # print("After removing")
  # print(topic_model_specter.topics)

  # defining path
  base_path = '/content/drive/MyDrive/NLP/'
  model_path = base_path + iter_version + beam_name + "/"
  create_path_if_not_exists(model_path)

  # save model as pickle file
  file_path = model_path + "model" + beam_name + ".pkl"
  joblib.dump(topic_model_specter, file_path) 


  # dynamic topic modeling
  years = input_data['pub_year'].to_list()
  topics_over_time = topic_model_specter.topics_over_time(list_text, topics, years)
  fig_time = topic_model_specter.visualize_topics_over_time(topics_over_time, top_n_topics=20)
  fig_name_png = model_path + "topic_time" + beam_name  +".png"
  fig_time.write_image(fig_name_png)
  fig_name_html = model_path + "topic_time" + beam_name  +".html"
  fig_time.write_html(fig_name_html)

  # visualize barchart of topics as html
  fig = topic_model_specter.visualize_barchart(top_n_topics = len(topic_model_specter.topics))
  fig_name = model_path + "bar_chart" + beam_name  +".html"
  fig_name_png = model_path + "bar_chart" + beam_name  +".png"

  fig.write_html(fig_name)
  fig.write_image(fig_name_png)


  # save as excel
  # df_info = pd.DataFrame(model.get_topic_info())
  excel_name = model_path + "Topic_Results.xlsx"
  # df_info.to_excel(excel_name,sheet_name="topic_info")

  df_topics = pd.DataFrame(topic_model_specter.topics)
  df_topics.to_excel(excel_name,sheet_name="topic_words")


  ## generating authors for each topic 
  df_authors, dict_authors = author_all_topics(topic_model_specter,input_data)
  # save df with authors 
  excel_name = model_path + "Author_Topics.xlsx"
  df_authors.to_excel(excel_name,sheet_name="authors_topics")

  search_term = input("Find authors and documents related to the term : ")


  similar_topics, similarity = topic_model_specter.find_topics(search_term, top_n=5)
  print("Authors are:",dict_authors[similar_topics[0]])

  # get all docs related to a particular search term 
  rep_docs = topic_model_specter.get_representative_docs(similar_topics[0])
  print("Closest documents are:",rep_docs)

  
  print("File: {} done ".format(i+1) )


print('Total time taken (mins): ', int((time.time()-start_time)/60))

